In [1]:
import gym
import sys
import random
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Dense

In [2]:
class DQN(tf.keras.Model):
    def __init__(self, state_size, action_size):
        super(DQN, self).__init__()
        self.fc1 = Dense(24,activation='relu')
        self.fc2 = Dense(24,activation='relu')
        self.out = Dense(action_size,kernel_initializer=tf.keras.initializers.RandomUniform(-1e-3,1e-3))

    def call(self,x):
        x = self.fc1(x)
        x = self.fc2(x)
        q = self.out(x)
        return q

In [3]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size= action_size
        
        # Hyper-parameters for learning
        self.discount_factor = 0.99
        self.learning_rate = 0.001
        self.epsilon = 1.0
        self.epsilon_decay = 0.999
        self.epsilon_min = 0.01
        
        # Experience Replay
        self.batch_size = 64
        self.train_start = 1000
        self.memory = deque(maxlen=2000)
        
        # Neural Network Architecture
        self.model        = DQN(self.state_size, self.action_size)
        self.target_model = DQN(self.state_size, self.action_size)
        self.optimizer = tf.keras.optimizers.Adam(lr=self.learning_rate)
        
        self.update_target_model()
        
    def remember(self, state, action, reward, next_state, done):
        next_state = np.reshape(next_state, [1, state_size])
        self.memory.append((state, action, reward, next_state, done))
        
    def update_target_model(self):
        self.target_model.set_weights(self.model.get_weights())
        
    def choose_action(self,state):
        # Exploration and Exploitation
        if (np.random.rand() <= self.epsilon):
            return random.randrange(self.action_size)
        else:
            return np.argmax(self.model.predict(state))
        
    def train_model(self, state, action, reward, next_state, done):
        # Train from Experience Replay
        # Training Condition - Memory Size
        if len(self.memory) < self.train_start:
            return 0.0
        # Decaying Exploration Ratio
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
        # Sampling from the memory
        mini_batch = random.sample(self.memory, self.batch_size)
        
        states      = np.array([sample[0][0] for sample in mini_batch])
        actions     = np.array([sample[1] for sample in mini_batch])
        rewards     = np.array([sample[2] for sample in mini_batch])
        next_states = np.array([sample[3][0] for sample in mini_batch])
        dones       = np.array([sample[4] for sample in mini_batch])
        
        model_params = self.model.trainable_variables
        with tf.GradientTape() as tape:
            q = self.model(states)
            one_hot_action = tf.one_hot(actions, self.action_size)
            q = tf.reduce_sum(one_hot_action * q, axis=1)
            
            target_q = self.target_model(next_states)
            target_q = tf.stop_gradient(target_q)
            
            max_q = np.amax(target_q, axis=-1)
            targets = rewards + (1 - dones) * self.discount_factor * max_q
            loss = tf.reduce_mean(tf.square(targets - q))
            
        grads = tape.gradient(loss, model_params)
        self.optimizer.apply_gradients(zip(grads, model_params))
        return loss

In [4]:
%matplotlib tk

ENV_NAME = 'CartPole-v1'
EPISODES = 1000
END_SCORE = 400

if __name__ == "__main__":
    env = gym.make(ENV_NAME)
    state_size = env.observation_space.shape[0]
    action_size = env.action_space.n

    agent = DQNAgent(state_size, action_size)
    print('Env Name : ',ENV_NAME)
    print('States {}, Actions {}'
            .format(state_size, action_size))

    scores, episodes, epsilons, losses = [], [], [], []
    score_avg = 0
    
    end = False
    
    for e in range(EPISODES):
        # Episode initialization
        done = False
        score = 0
        loss_list = []

        state = env.reset()
        
        while not done:
            #env.render()

            # Interact with env.
            action = agent.choose_action(state)
            state = np.reshape(state, [1, state_size])
            next_state, reward, done, info = env.step(action)
            agent.remember(state, action, reward, next_state, done)
            loss = agent.train_model(state, action, reward, next_state, done)
            state = next_state

            # 
            score += reward
            loss_list.append(loss)
            if done:
                agent.update_target_model()

                score_avg = 0.9 * score_avg + 0.1 * score if score_avg != 0 else score
                print('epi: {:3d} | score avg {:3.2f} | mem length: {:4d} | epsilon: {:.4f}'
                      .format(e, score_avg, len(agent.memory), agent.epsilon))

                # Save data for plot
                episodes.append(e)
                scores.append(score_avg)
                epsilons.append(agent.epsilon)
                losses.append(np.mean(loss_list))

                # View data
                plt.subplot(311)
                plt.plot(episodes, scores, 'b')
                plt.xlabel('episode')
                plt.ylabel('average score')
                plt.title('cartpole DQN')
                plt.grid()
                
                plt.subplot(312)
                plt.plot(episodes, epsilons, 'b')
                plt.xlabel('episode')
                plt.ylabel('epsilon')
                plt.grid()
                
                plt.subplot(313)
                plt.plot(episodes, losses, 'b')
                plt.xlabel('episode')
                plt.ylabel('losses')
                plt.grid()
                
                plt.savefig('./save_model/cartpole_dqn_TF.png')

                if score_avg > END_SCORE:
                    agent.model.save_weights('./save_model/cartpole_dqn_TF', save_format='tf')
                    end = True
                    break
        if end == True:
            env.close()
            np.save('./save_model/data/cartpole_dqn_TF_epi',  episodes)
            np.save('./save_model/data/cartpole_dqn_TF_score',scores)
            np.save('./save_model/data/cartpole_dqn_TF_loss', losses)
            print("End")
            break

Env Name :  CartPole-v1
States 4, Actions 2
epi:   0 | score avg 20.00 | mem length:   20 | epsilon: 1.0000
epi:   1 | score avg 19.60 | mem length:   36 | epsilon: 1.0000
epi:   2 | score avg 19.94 | mem length:   59 | epsilon: 1.0000
epi:   3 | score avg 20.55 | mem length:   85 | epsilon: 1.0000
epi:   4 | score avg 19.99 | mem length:  100 | epsilon: 1.0000
epi:   5 | score avg 19.59 | mem length:  116 | epsilon: 1.0000
epi:   6 | score avg 19.23 | mem length:  132 | epsilon: 1.0000
epi:   7 | score avg 18.31 | mem length:  142 | epsilon: 1.0000
epi:   8 | score avg 17.78 | mem length:  155 | epsilon: 1.0000
epi:   9 | score avg 18.90 | mem length:  184 | epsilon: 1.0000
epi:  10 | score avg 19.01 | mem length:  204 | epsilon: 1.0000
epi:  11 | score avg 18.81 | mem length:  221 | epsilon: 1.0000
epi:  12 | score avg 19.43 | mem length:  246 | epsilon: 1.0000
epi:  13 | score avg 19.89 | mem length:  270 | epsilon: 1.0000
epi:  14 | score avg 20.00 | mem length:  291 | epsilon: 1.0